### Chargement des modules nécessaires pour le projet
Commencez par exécuter la cellule ci-dessous. Elle va charger tout le nécessaire pour faire fonctionner votre code. 
Si vous le lancez en ligne, __soyez patients__ (attendez que l'étoile à gauche devienne un nombre entre crochets).


In [1]:
include("jupy.jl")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


    Fetching:

    Fetching: [==============>                          ]  

Fetching: [========================================>]  100.0 %              ]  61.2 %==============================>          ]  73.9 %

  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


main (generic function with 1 method)

### Donnéees à utiliser

On rappelle les structures qui sont remplies par les méthodes de lecture de données.

#### data structure for data used to compute the balance cost
```
struct BalanceData
    resource1::Int64        # first resource
    resource2::Int64        # second resource
    target::Int64           # target aimed for the recource consumption
    weight::Int64           # weight of the balance cost
end
```

#### data struture for an instance of Google challenge problem
```
struct DataGoogle
    nbResources::Int64               # number of resources in the instance
    transientStatus::Array{Int64}    # table indicating if each resource is transient (O/1)
    weightLoadCost::Array{Int64}     # table indicating the weight of load cost for each resource

    nbMachines::Int64                # number of machines
    neighborhoods::Array{Int64}      # table indicating the neighborhood of each machine
    nbNeighborhoods::Int64           # total number of neighbordhoods
    locations::Array{Int64}          # table indicating the location of each machine
    nbLocations::Int64               # total number of locations
    softCapacities::Array{Int64,2}   # softcapacities[m,r] indicates the soft cap of machine m for resource r
    hardCapacities::Array{Int64,2}   # hardcapacities[m,r] indicates the hard cap of machine m for resource r
    machineMoveCosts::Array{Int64,2} # machineMoveCosts[m1,m2] indicates the cost for moving from m1 to m2

    nbServices::Int64                # number of services
    spreadMin::Array{Int64}          # for each service, its spread min
    dependences::Array{Array{Int64}} # for each service, a list of dependences (service numbers)

    nbProcess::Int64                 # number of processes
    servicesProcess::Array{Int64}    # for each process  its service
    processReq::Array{Int64,2}       # processReq[p,r] indicates the consumption of resource r by p
    processMoveCost::Array{Int64}    # cost of moving each process p

    nbBalanceCostData::Int64               # number of balance constraints
    balanceCostDataList::Array{BalanceData} # one for each balance target
    processMoveWeight::Int64        # weight for the process move cost
    serviceMoveWeight::Int64        # weight for the service move cost
    machineMoveWeight::Int64        # weight for the machine move cost

    initialAssignment # for each process, id of the machine in the initial solution
end
```

#### slution 
```
struct SolutionGoogle
    assignment::Array{Int64}
    cost::Int64
end
```





### Cellule à modifier
Insérez ci-dessous votre modèle dans la méthode **solveGoogle**. 

N'oubliez pas de faire un run sur la cellule quand vous avez fini. 

In [4]:
using Cbc
using JuMP
function solveGoogle(data::DataGoogle, verbose::Bool)
    # model 
    m = Model(Cbc.Optimizer)    # on crée un modèle en précisant qu'on va utiliser Cbc
    
    @variable(m,x[i=1:data.nbMachines,j=1:data.nbProcess],Bin) # Process j affecté à machine i (xij is Binary)

    println("---------Nbre machines----------")
    println(data.nbMachines)
    println("---------Nbre processus---------")
    println(data.nbProcess)
    
    println("--------------------------------")
    
    @variable(m,moved[i=1:data.nbProcess],Bin) # if process i was moved
    
    #@objective(m, Min, sum(data.processMoveCost[i]*moved[i] for i=1:data.nbProcess) ) #O3
    #@objective(m, Min, sum( data.weightLoadCost[r] * sum( (sum(data.processReq[j,r]*(x[i,j]) for j=1:data.nbProcess) - data.softCapacities[i,r] ) for i=1:data.nbMachines if (sum(data.processReq[j,r]*(x[i,j]) for j=1:data.nbProcess) >= data.softCapacities[i,r]) ) for r=1:data.nbResources)) #O1
    
    function U(m,r)
        sum=0
        for j in 1:data.nbProcess
            sum+=data.processReq[j,r]*(x[m,j])
        end
        return sum
    end
    function SC(m)
        sum=0
        for i in 1:data.nbResources
            sum+=data.softCapacities[m,i]
        end
        return sum#sum(data.softCapacities[m,r] for r=1:data.nbResources)
    end
    @variable(m,MAX,Bin) # MAX 0 et u - sc
    @variable(m,isMAX,Bin) # aide à calculer MAX
    function loadCost(r)
        u=0
        sc=0
        somme = 0
        for i=1:data.nbMachines
            for j in 1:data.nbResources
                sc+=data.softCapacities[i,j]
            end
            
            for k in 1:data.nbProcess
                u+=data.processReq[k,r]*(x[i,k])
            end
            
            #if u - sc >= 1  # le max { U(m,r) - SC(m,r) }
            #    somme += u - sc #U(m,r) - SC(m,r)
            #end
            @constraint(m,MAX >= 0)
            @constraint(m,MAX >= u - sc)
            @constraint(m,MAX <= 0 + 10^10 * isMAX)
            @constraint(m,MAX <= u - sc + 10^10 * (1 - isMAX))
            somme+= MAX
        end
        return somme
    end
    @objective(m, Min, sum(loadCost(r) for r=1:data.nbResources)) #on doit multiplier par le weight
    
    #@constraint(m,co[j=1:data.nbProcess];data.initialAssignment[j]==sum(p*x[j,p] for p=1:data.nbProcess),moved[j] == 0 )
    
    
    @constraint(m, constraint1_1[j=1:data.nbProcess], sum(x[i,j] for i=1:data.nbMachines) == 1 ) # C1
    
    @constraint(m, constraint2[i=1:data.nbMachines,r=1:data.nbResources], sum(data.processReq[j,r]*(x[i,j]) for j=1:data.nbProcess) <= data.hardCapacities[i,r] ) # C2
    
    ###################### Contrainte 3 ######################################
    for p1 in 1:data.nbProcess
        for p2 in 1:data.nbProcess
            if data.servicesProcess[p1]==data.servicesProcess[p2]
                if p1 != p2
                    for i in 1:data.nbMachines
                        @constraint(m, x[i,p1]+x[i,p2] <= 1 )
                    end
                end
            end
        end
    end
    
    print(m)        # affichage du modèle
    optimize!(m)    # resolution à l'aide du solveur

    objVal = JuMP.objective_value(m) # to be replaced by the objective value 
    soluce = Array{Int64}(undef, data.nbProcess) # create an empty table
    #sum(k*JuMP.value(x[k,i]) for k=1:data.nbMachines)
    for i in 1:data.nbProcess
        soluce[i] = sum(k*round(JuMP.value(x[k,i])) for k=1:data.nbMachines)  # to be replaced by the assignment found by your model 
    end
    return SolutionGoogle(soluce,objVal)
end

LoadError: syntax: extra token "for" after end of expression

### Cellule de test
La cellule ci-dessous permet d'exécuter le code. Elle lit la donnée, lance votre méthode solveGoogle, récupère la solution, et vérifie le respect des contraintes et son coût. 

Vous pouvez modifier le nom pour tester sur un autre jeu de donnée. 

In [3]:
# enter your instance name here 
donnee = "a1_1"  # possibles values a1_1, a1_2, a1_3, a1_4, a1_5, a2_1, a2_2, a2_3, a2_4, a2_5 

# set to yes if you want to print intermediate logs 
verbose = false

main("model_"*donnee*".txt","assignment_"*donnee*".txt",verbose)


Received instance files model_a1_1.txt and assignment_a1_1.txt


UndefVarError: UndefVarError: solveGoogle not defined